# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [46]:
# !pip install selenium

In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Passo 1: Pegar cotação do dólar
navegador = webdriver.Chrome()
navegador.get("https://www.google.com.br/")
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').click()
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# Passo 2: Pegar cotação do euro
navegador.get("https://www.google.com.br/")
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').click()
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 3: Pegar cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')

cotacao_ouro = cotacao_ouro.replace(',','.')

print(cotacao_ouro)

navegador.quit()

5.1800999999999995
5.2879
291.99


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [48]:
import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,$999.99,Dólar,5,"$4,999.95",$1.40,"$6,999.93"
1,Carro Renault,"$4,500.00",Euro,6,"$27,000.00",$2.00,"$54,000.00"
2,Notebook Dell,$899.99,Dólar,5,"$4,499.95",$1.70,"$7,649.91"
3,IPhone,$799.00,Dólar,5,"$3,995.00",$1.70,"$6,791.50"
4,Carro Fiat,"$3,000.00",Euro,6,"$18,000.00",$1.90,"$34,200.00"
5,Celular Xiaomi,$480.48,Dólar,5,"$2,402.40",$2.00,"$4,804.80"
6,Joia 20g,$20.00,Ouro,350,"$7,000.00",$1.15,"$8,050.00"


- Atualizando os preços e o cálculo do Preço Final

In [49]:
# Passo 4: Atualizar a base de dados

tabela.loc[tabela["Moeda"]=="Dólar","Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"]=="Euro","Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro)


# Passo 5: Recalcular os preços

# Preço de Compra = Preço Original * Cotação

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Preço de Venda = Preço de Compra * Margem

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display (tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,$999.99,Dólar,$5.18,"$5,180.05",$1.40,"$7,252.07"
1,Carro Renault,"$4,500.00",Euro,$5.29,"$23,795.55",$2.00,"$47,591.10"
2,Notebook Dell,$899.99,Dólar,$5.18,"$4,662.04",$1.70,"$7,925.46"
3,IPhone,$799.00,Dólar,$5.18,"$4,138.90",$1.70,"$7,036.13"
4,Carro Fiat,"$3,000.00",Euro,$5.29,"$15,863.70",$1.90,"$30,141.03"
5,Celular Xiaomi,$480.48,Dólar,$5.18,"$2,488.93",$2.00,"$4,977.87"
6,Joia 20g,$20.00,Ouro,$291.99,"$5,839.80",$1.15,"$6,715.77"


### Agora vamos exportar a nova base de preços atualizada

In [50]:
# Passo 6: Exportar a base de dados atualizada

tabela.to_excel("Produtos Novo.xlsx", index=False)